In [1]:
import numpy as np

import sp_model 
from util import to_range, DATA_PATH, FIG_PATH, OptimizationMethod
from sp_model import SET, PARAMETER

# import gurobipy as gp
# with gp.Env(empty=True) as env:
#     env.setParam('OutputFlag', 0)
#     env.start()
#     with gp.Model(env=env) as m:
# %%
# 有一個趨勢是幾乎每個點都會開立一個rdc, cs，很像沒有不開的。
m, obj1, obj2 = sp_model.solve(1, OptimizationMethod.WEIGHTED_SUM)
obj1_star = obj1.getValue() 
m, obj1, obj2 = sp_model.solve(0, OptimizationMethod.WEIGHTED_SUM)
obj2_star = obj2.getValue()
objstars = [obj1_star, obj2_star]

d = dict()
w = 0.1

# for w in np.arange(0.01, 0.2, 0.01):
spm, o1, o2 = sp_model.solve(w,
            OptimizationMethod.LP_METRIC,
            objstars)
alpha, beta = [], [] 
for j in to_range(SET['J']):
    alpha_j = spm.getVarByName(f'alpha[{j}]').x
    beta_j = spm.getVarByName(f'beta[{j}]').x
    alpha.append(alpha_j)
    beta.append(beta_j)
    d[str(w)] = {'#RDC': sum(alpha), '#CS': sum(beta)}
# soldict = {} 
# for rdc_num in to_range(SET['J']):
#     eps = [rdc_num, 15 - rdc_num - 1] 
#     model = sp_model.solve(w,  
#         OptimizationMethod.WEIGHTED_SUM, 
#         eps = eps)
#     try: 
#         print(model.objVal) 
#         soldict[rdc_num]
#     except: 
#         print('infeasible')
#         pass 
    

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-15
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 4637 rows, 4632 columns and 13095 nonzeros
Model fingerprint: 0x46b74312
Model has 666 quadratic constraints
Model has 12 general constraints
Variable types: 4422 continuous, 210 integer (210 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+13]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [3e-01, 1e+00]
  Objective range  [2e-03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
  QRHS range       [7e+00, 2e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

-------------------------------------------------------------

In [2]:
for j in to_range(SET['J']):
   for c in to_range(SET['C']):
    for s in to_range(SET['S']): 
        delta = spm.getVarByName('delta[0,0,0]').x
        if delta != 0 :
            print(f'j={j}, c={c}, s={s}, delta={delta}')

In [3]:
# get supplier names 
import pandas as pd 
PATH_PREFIX = 'MoDRL_'
df_supplier = pd.read_csv(DATA_PATH 
+ f'/{PATH_PREFIX}supplier.csv')
suppliers = df_supplier['Suppliers'].values 

df_distance = pd.read_csv(DATA_PATH + f'/{PATH_PREFIX}distance.csv')
AA = df_distance.columns.values
J = AA
print(suppliers)
print(AA)
Commodities = ['Water', 'Food', 'Shelter']

['Sari' 'Qazvin' 'Tehran' 'Arak' 'Isfahan']
['Gorgan' 'Semnan' 'Sari' 'Rasht' 'Qazvin' 'Karaj' 'Tehran' 'Varamin'
 'Robatkarim' 'Islamshahr' 'Shahriar' 'Gom' 'Arak' 'Isfahan' 'Kashan']


In [4]:
# from sp_model import PARAMETER
# S = PARAMETER['S']
# print(S) # capacity of each supplier for each commodity 

### $\alpha_j, \beta_j$ Build RDC/CS or not 

In [6]:
from sp_model import SET, PARAMETER
alpha, beta = [], [] 
for j in to_range(SET['J']):
    alpha_j = spm.getVarByName(f'alpha[{j}]').x
    beta_j = spm.getVarByName(f'beta[{j}]').x
    # alpha_i: build RDC or not 
    # beta_i: build CS or not
    alpha.append(alpha_j)
    beta.append(beta_j)
    # print(AA[i], alpha_i, beta_i)
build_df = pd.DataFrame({'RDC': alpha, 'CS': beta}, index=AA)
build_df = build_df.transpose() 
build_df

,Gorgan,Semnan,Sari,Rasht,Qazvin,Karaj,Tehran,Varamin,Robatkarim,Islamshahr,Shahriar,Gom,Arak,Isfahan,Kashan
RDC,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
CS,-0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0


In [7]:
print(f'RDC #: {sum(alpha)}')
print(f'CS #: {sum(beta)}')

RDC #: 7.0
CS #: 8.0


In [8]:
i, j, k, k_prime, s, c = [len(idx) for idx in SET.values()]
import numpy as np 
Q = np.zeros((i, j, c))
X = np.zeros((i, j, c, s))
Y = np.zeros((j, k, c, s))

### $Q_{ijc}, X_{ijcs}$ transported commodities to RDC/CS

In [9]:
## in preparedness phase 
## get Q_ijc: Sup to RDC/CS
for i in to_range(SET['I']):
    for j in to_range(SET['J']):
        for c in to_range(SET['C']):
            Q[i, j, c] = spm.getVarByName(f'Q[{i},{j},{c}]').x
            if Q[i, j, c] > 0: 
                print(f'Supplier {suppliers[i]} => {J[j]} {Commodities[c]} : {Q[i, j, c]}') 

In [ ]:
## response phase 
#  get X_ijcs: Sup to RDC/CS in response phase 
print('======= RDC =======')
for i in to_range(SET['I']):
    for j in to_range(SET['J']):
        for c in to_range(SET['C']):
            s_vec = np.zeros(len(SET['S']))
            for s in to_range(SET['S']):
                X[i, j, c, s] = spm.getVarByName(f'X[{i},{j},{c},{s}]').x
                s_vec[s] = X[i, j, c, s] 
            if np.sum(s_vec) != 0 and alpha[j]: 
                print(f'{suppliers[i]} => {AA[j]}', Commodities[c], s_vec)

======= RDC =======


In [10]:
print(f'======= CS =======')
for i in to_range(SET['I']):
    for j in to_range(SET['J']):
        for c in to_range(SET['C']):
            s_vec = np.zeros(len(SET['S']))
            for s in to_range(SET['S']):
                X[i, j, c, s] = spm.getVarByName(f'X[{i},{j},{c},{s}]').x
                s_vec[s] = X[i, j, c, s] 
            if np.sum(s_vec) != 0 and beta[j]:
                s_vec = tuple(s_vec) 
                print(f'{suppliers[i]} => {AA[j]}', Commodities[c], s_vec)


======= CS =======
Sari => Sari Water (0.0, 117.0, 117.0, 117.0)
Sari => Sari Food (117.0, 117.0, 117.0, 117.0)
Sari => Sari Shelter (0.0, 0.0, 39.0, 39.0)
Qazvin => Tehran Food (55.599999999999966, 0.0, 0.0, 0.0)
Qazvin => Shahriar Water (0.0, 117.0, 117.0, 117.0)
Qazvin => Shahriar Food (61.400000000000034, 117.0, 117.0, 117.0)
Qazvin => Shahriar Shelter (0.0, 39.0, 39.0, 39.0)
Tehran => Tehran Water (55.0, 0.0, 0.0, 0.0)
Tehran => Tehran Food (132.6, 0.0, 0.0, 0.0)
Tehran => Islamshahr Water (0.0, 132.6, 132.6, 132.6)
Tehran => Islamshahr Food (0.0, 132.6, 132.6, 132.6)
Tehran => Islamshahr Shelter (0.0, 44.2, 44.2, 44.2)
Arak => Tehran Food (117.0, 0.0, 0.0, 0.0)
Arak => Islamshahr Water (0.0, 117.0, 117.0, 117.0)
Arak => Islamshahr Food (0.0, 117.0, 117.0, 117.0)
Arak => Islamshahr Shelter (0.0, 39.0, 39.0, 39.0)
Isfahan => Islamshahr Food (0.0, 0.0, 117.0, 0.0)
Isfahan => Isfahan Water (0.0, 117.0, 117.0, 117.0)
Isfahan => Isfahan Food (117.0, 117.0, 0.0, 117.0)
Isfahan => Isfaha

### $Y_{jkcs}$ transported commodities to AA

In [ ]:
    
def isNotZero(x):
    return abs(x) > 1e-6

for j in to_range(SET['J']):
    for k in to_range(SET['K']):
        for c in to_range(SET['C']):
            # s_vec contains (s1, s2, s3, s4) 
            s_vec = np.zeros(len(SET['S']))
            for s in to_range(SET['S']):
                Y[j, k, c, s] = spm.getVarByName(f'Y[{j},{k},{c},{s}]').x
                s_vec[s] = Y[j, k, c, s]
            if sum(s_vec) > 0 and (alpha[j] or beta[j]):
                # J and K share the same node set
                if alpha[j]:
                    identity = 'RDC'
                elif beta[j]:
                    identity = 'CS'
                else: 
                    identity = 'None'
                
                print(identity)
                print(f'{J[j]} => {AA[k]}', Commodities[c], s_vec)

CS
Sari => Gorgan Food [45.2  0.   0.   0. ]
CS
Sari => Rasht Food [71.8  0.   0.   0. ]
CS
Sari => Tehran Water [  0. 117. 117. 117.]
CS
Sari => Tehran Food [  0. 117. 117. 117.]
CS
Sari => Tehran Shelter [ 0. 39. 39. 39.]
RDC
Karaj => Arak Water [ 0.  0. 75.  0.]
RDC
Karaj => Arak Food [ 0.  0. 75.  0.]
CS
Tehran => Sari Water [93.8  0.   0.   0. ]
CS
Tehran => Sari Food [305.2   0.    0.    0. ]
CS
Tehran => Rasht Water [38.8  0.   0.   0. ]
CS
Islamshahr => Tehran Water [  0.  249.6 249.6 249.6]
CS
Islamshahr => Tehran Food [  0.  249.6 366.6 249.6]
CS
Islamshahr => Tehran Shelter [ 0.  83.2 83.2 83.2]
CS
Shahriar => Rasht Food [61.4  0.   0.   0. ]
CS
Shahriar => Tehran Water [  0. 117. 117. 117.]
CS
Shahriar => Tehran Food [  0. 117. 117. 117.]
CS
Shahriar => Tehran Shelter [ 0. 39. 39. 39.]
RDC
Arak => Robatkarim Food [  0.   0. 100.   0.]
CS
Isfahan => Rasht Food [117.   0.   0.   0.]
CS
Isfahan => Tehran Water [  0. 117. 117. 117.]
CS
Isfahan => Tehran Food [  0. 117.   0. 117

### $I_{kcs}$: the inventory held at AA k for commod c under s 
### $b_{kcs}$: the shortage held at AA k for commod c under s  

In [ ]:
k, c, s = len(SET['K']), len(SET['C']), len(SET['S'])
I = np.zeros((k, c, s))
b = np.zeros((k, c, s))

for s in to_range(SET['S']):
    print('s=', s)
    for k in to_range(SET['K']):
        for c in to_range(SET['C']):
            
            I[k, c, s] = spm.getVarByName(f'I[{k},{c},{s}]').x
            b[k, c, s] = spm.getVarByName(f'b[{k},{c},{s}]').x
            
            if isNotZero(I[k, c, s]):
                print(f'inventory: AA {k}', Commodities[c], I[k, c, s])
            if isNotZero(b[k, c, s]):
                print(f'shortage: AA {k}', Commodities[c], b[k, c, s])
                

s= 0
inventory: AA 0 Water 166.2
shortage: AA 0 Water 485.2
shortage: AA 0 Food 273.8
shortage: AA 0 Shelter 106.0
inventory: AA 1 Water 451.2
shortage: AA 1 Water 485.2
shortage: AA 1 Food 34.0
shortage: AA 1 Shelter 11.0
shortage: AA 2 Water 485.2
shortage: AA 2 Food 273.8
shortage: AA 2 Shelter 193.0
shortage: AA 3 Water 485.2
shortage: AA 3 Food 273.8
shortage: AA 3 Shelter 175.0
inventory: AA 4 Water 417.2
shortage: AA 4 Water 485.2
shortage: AA 4 Food 68.0
shortage: AA 4 Shelter 23.0
s= 1
inventory: AA 0 Water 658.4000000000001
shortage: AA 0 Water 817.4000000000001
inventory: AA 0 Food 658.4
shortage: AA 0 Food 817.4
shortage: AA 0 Shelter 53.0
shortage: AA 1 Water 69.0
inventory: AA 1 Food 748.4
shortage: AA 1 Food 817.4
shortage: AA 1 Shelter 23.0
inventory: AA 2 Water 296.4000000000001
shortage: AA 2 Water 817.4000000000001
inventory: AA 2 Food 296.4
shortage: AA 2 Food 817.4
shortage: AA 2 Shelter 174.0
inventory: AA 3 Water 388.4000000000001
shortage: AA 3 Water 817.4000000